# Genome Assembly with Redbean (Wtdbg2)

[Wtdbg2 aka Redbean](https://github.com/ruanjue/wtdbg2) "is a de novo sequence assembler for long noisy reads produced by PacBio or Oxford Nanopore Technologies (ONT). It assembles raw reads without error correction and then builds the consensus from intermediate assembly output."

Also according to their documentation, "wtdbg2 chops reads into 1024bp segments, merges similar segments into a vertex and connects vertices based on the segment adjacency on reads. The resulting graph is called fuzzy Bruijn graph (FBG). It is akin to De Bruijn graph but permits mismatches/gaps and keeps read paths when collapsing k-mers. The use of FBG distinguishes wtdbg2 from the majority of long-read assemblers."

## What you'll need to run this notebook

1. You will need a set of sequencing reads produced previously (i.e. from running your `ReadQC-with-fastp` notebook). 
2. We recommend running this notebook with at least 32 CPUs and 64GB of RAM - genome assembly is usually computationally intensive. Some programs may take days even on a very powerful machine. 

### Watch the video introduction for a little bit of background on how this assembly tool works

[WTDBG2: Long Read Genome Assembly Tool](https://youtu.be/5aHEAzanm5k)

## Installing Redbean

As always, we will install the software; again we will use conda. 

**Important**: Make sure you execute each numbered step 

1. We will search for the tool we want to install

We will use the `conda search` command and the channel (`-c`) flag to search [bioconda](https://bioconda.github.io/)

In [ ]:
conda search wtdbg -c bioconda

2. Create a conda enviornment

Conda uses something called "enviornments" which are essentially isolated configurations on our computer where we can included all the needed compatible tools and exlude other tools which are unnesessary or would have conflicts with our desired tool. We will use the `-y` option to install without prompting the user for input, the `--name` option to name the enviornment for the tool. We will enforce versioning (`tool==version`) so that we know what version of a tool was used to do an analysis should we wish to repeat the analysis. 

**Tip**: Use the latest version where possible, but if you get an error with dependancies, using a lower version may help. Some tools may never be installed successfully using conda, but we will face those when we have too. 

**Bonus tip**: In the installation command below we also specify the [build](https://medium.com/webgentle/what-is-the-software-build-all-you-need-to-know-4046b0e674bb) by adding an additional `=` after the version (2.5) and copying from the build column from our search results  above. 


In [ ]:
conda create -y --name redbean wtdbg==2.5=h8b12597_0 -c bioconda

3. We will use the `conda init` command so that conda can be configured for this shell

In [ ]:
conda init

4. **DON'T SKIP**: We need to restart the computer's [kernal](https://en.wikipedia.org/wiki/Kernel_(operating_system)). Go to the **Kernal** menu and choose **Restart Kernal**

5. Finally, we can activate the conda enviornment (created with the name used for the environment). When you run the next cell it should return the name of the environment.  

In [ ]:
conda activate redbean

## Running Redbean

Previously, we ran our assembly on a small subset of reads (100 or 1000 reads). Here we may as well run on the entire set of reads from your quality control step. 

**Tip**: When using commands or searching for files, the tab key will help you autocomplete (and help ensure the files and commands you think you have are actually accessible).



## Questions to answer before running this software

It's important you know these answers. You will likely do more than one assembly so that you can tweak the settings and make improvements. You may need to refer to your `fastp` output to answer these. 

1. What is the name of the file containing your cleaned reads
2. How many reads total do you have in your input? 
3. How many nucleotides do you have in your input? 
4. What is the mean length of your reads?
5. What percentage of reads are Q20 ([phred score](https://en.wikipedia.org/wiki/Phred_quality_score)) or above?
6. What is your estimated coverage?

To answer question #6 remember, coverage is how many nucleotides you have/the size (also in nucleotides) of the genome. For example, if you have 5 Gigabases (5GiB, 5,000,000,000 bases) and your genome is 150Mb, 150,000,000 you have ~ 33X coverage (33 times the genome size). 

### Redbean options

When we call the `wtdbg2` (Redbean) program here are some options we are likely to use (see the [Redbean](https://github.com/ruanjue/wtdbg2) documentation for all options):


- `-x`: Specify the sequencing technology, in our case `ont` for Oxford Nanopore
- `-i`: Specify the file containing  the input FastQ reads
- `-o`: Specify the output directory/prefix
- `-g`: Estimated genome size in millions (m) or billions (g) of nucleotides (e.g. 5m or 2.6g)
- `-t`: How many CPUs to use where int= some integer (e.g. 32)

**Tip**: Use the help command to see all options

In [ ]:
wtdbg2 --help

### Example 1 (optional) - running readbean on `spolyrhiza_reads_filtered.fastq.gz`

The file `spolyrhiza_reads_filtered.fastq.gz` was generated in advance using the raw reads (`spolyrhiza_reads.fastq.gz`) cleaned and flitered with `fastp`. The resulting data set has the following characteristics: 

- Total reads: 1.06 million
- Total bases: 6.57 billion 
- Q20 bases: 4.39 billion
- Mean read length: 6149bp

If you like, you can run `wtdbg2` on this dataset (since we know it works and will generate an assembly). The following command is a good example to show you how to run `wtdbg2`. You can modify it to run with your own input file. 

**Tip**: The *S.polyrhiza* genome size is about 150 million bases

1. First, since we will have a lot of results, let's make an output directory first to keep results organizes

In [ ]:
mkdir -p data/output/redbean_example_assembly

2. Redbean has two steps to generate the assembly. First reads are assembled into a "contig layout" and another command is used to "condense" those contigs into the final consensus in FASTA. 

First we run the command below to assemble (this will take **several** hours)

**IMPORTANT - how to know when this is finnished**: The [\*] you see next to a cell indicates that a program is running. If you see this, the program is still running (even though a given stage of the program may say "DONE"). 

Wait for a message that says "Program  Done" such as:

`[Wed Dec 12 03:22:05 2021] Program Done`

In [ ]:
wtdbg2 -o data/output/redbean_example_assembly/redbean_example \
       -i data/input/concat_fastq/spolyrhiza_reads_filtered.fastq.gz \
       -x "ont" \
       -t 40 \
       -g "150m"

3. Next we will run the condenser step on the intermediate files produced in the assembly above. (This will also take time. Will end with a statement like:

`** PROC_STAT(TOTAL) **: real 1380.702 sec, user 53941.380 sec, sys 375.300 sec, maxrss 8737888.0 kB, maxvsize 11322640.0 kB`

In [ ]:
wtpoa-cns -t 40\
          -i data/output/redbean_example_assembly/redbean_example.ctg.lay.gz\
          -o data/output/redbean_example_assembly/spolyrhiza_wtdbg.ctg.fa

3. In your Jupyter file browser, navigate to the home folder (See the home icon) and see your results in the `readbean_example_assembly` folder. 

### Results

Like most assemblers, at each step Redbean (`wtdbg2`) will generate several intermediate outputs. In the output folder, you will see `spolyrhiza_wtdbg.ctg.fa` (this is what we named using the `-o` output above). Some interesting results from the above report include the length of the genome assembled (~138.1 Mb), ~601 contigs (fewer is better) and the [N50](https://www.metagenomics.wiki/pdf/definition/assembly/n50) of 5,474,149 bp (higher is better); note that each assembly will be slightly different - even with the same settings and input data. Can you tweak the assembly to get fewer fragments ([contigs](https://en.wikipedia.org/wiki/Contig)) or higher N50?

The assembly you generate can now be evaluated in the next stage where we will use `quast` to generate metrics we can use to estimate the quality of this assembly.  

## Challenge - Use Redbean to assemble your reads

Now, it's up to you to use  `wtdbg2` to generate your own genome assembly using the reads generated by fastp. 


### What to do

1. Use the `mkdir` command to make a unique output directory to save your results (e.g. redbean_assembly_1, redbean_assembly_2). Make a new folder each time you do a new assembly. 

In [ ]:
mkdir 

2. Complete the command below to try an assembly using the same parameters as the example above. You will have to adjust the file path/name (`-i`) to match your file path/name, specify the output directory (`-o`), and adjust the threads to the number of CPUs you are using (`-t`); CPU number was specified when you launched the application in the Discovery Environment. 

**Note about completing the cells below**
- Enter your options after the flag (e.g. `-o`) and before the `\`
- Leave a space between what you write and the `\`
- Look closely how the working commands in the example are written if you have difficulties
- Look closely for errors (unable to find file,  unable to write file, etc.)
- Ask for help if you can't get something to work

In [ ]:
wtdbg2 -o \
       -i  \
       -x "ont" \
       -t  \
       -g "150m"

You also need to run the consenser tool. Your threads (`-t`) should be the same. The input (`-i`) should be the file created in the folder above and should end with a `.ctg.lay.gz` extension. Your output  (`-o`) should end  with a `.ctg.fa` extension. 

In [ ]:
wtpoa-cns -t 40\
          -i \
          -o 

3. Things to change. There are many things you can change which may improve (or worsen) the quality of your assembly. For now, it does not make sense to try too many things, but we suggest making at least one change so that when you evaluate your assembly in the next notebook, you already have some things to look at. 




## Try something

Generate another assembly to see what parameters make a difference. 

Possible changes to try

- You were asked to create two different `fastp` outputs, try them both with the same settings
- You can adjust the [kmer](https://en.wikipedia.org/wiki/K-mer) size used by the software. Redbean, like many alignment software programs chops the sequence into smaller bits called "k-mers" where k is some odd number (e.g. 17, 19, ...). If  you look at the `wtdbg2 --help` command, when we choose the settings for nanopore data (`-x "ont"`) we choose a k of 15. You can add a new option (`-k`) and change it to a new number (from 0-23). Adjusting slightly up or down (e.g. `-k 13` or `-k 17`) may or may not improve your assembly (fewer, longer contigs). The only way to find out is to try!

Add as many cells as you need to try alternative assembles. You should also try at least one other assembler. 

## Document your work

We need to keep good track on what changes were made/how a file was produced so that we can fully document our work. In this exercise, it will be critical to know your settings so we can compare results across everyone who does this experiment. You will also be able to go back and reproduce your work if needed. 

**Make sure to save a copy of this notebook**

When you terminate your application in CyVerse the results and data should be written back. You can also select this notebook in the file browser and choose Save and Export Notebook As (HTML) to save an easy-to-read version you can view anytime. 